In [ ]:
import sys
print(sys.executable)

In [ ]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd

Setting up environment variables for OpenAI and AstraDB

In [ ]:
load_dotenv()

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")

In [ ]:
os.environ["ASTRA_DB_API_ENDPOINT"] = ASTRA_DB_API_ENDPOINT
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = ASTRA_DB_APPLICATION_TOKEN

Created an instance of OpenAIEmbeddings and AstraDBVectorStore.

In [ ]:
embedding = OpenAIEmbeddings()

In [ ]:
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="financebot",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace="default_keyspace"    
)

Create a list of documents from PDF.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
%pwd

In [ ]:
%cd ..

In [ ]:
file_path = "data/finance_data.pdf"
loader = PyPDFLoader(file_path)

In [ ]:
pages = loader.load()
len(pages)

In [ ]:
pages = pages[10:20]

In [ ]:
for page in pages:
    print(page.page_content)

In [ ]:
raw_text = ""
for i , page in enumerate(pages):
    raw_text += page.page_content

print(raw_text)

Create chunks of text from pages created above

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)



In [ ]:
texts = text_splitter.split_text(raw_text)
len(texts)

Number of chunks created: 90

In [ ]:
type(texts)

In [ ]:
type(texts[0])

Converting string to Document type before pushing to vector store

In [ ]:
from langchain.docstore.document import Document

In [ ]:
docs = [Document(page_content = text) for text in texts]

In [ ]:
docs[0]

Store the vectors in the AstraDB vector store

In [ ]:
vstore.add_documents(docs)

Create a retriever using the AstraDB vector store

In [ ]:
retriever = vstore.as_retriever(search_kwargs={"k":3}) # k is the number of documents to return

Example of how retriever works

In [ ]:
retriever.get_relevant_documents("what is Market For Registrant’s Common Equity?")

Create a prompt template for the LLM model

In [ ]:
FINANCE_BOT_TEMPLATE = """" 
    Your finance bot is an expert in finance related advice.
    Ensure your answers are relevant to the query context and refrain from straying off-topic.
    Your responses should be concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(FINANCE_BOT_TEMPLATE)

Instantiate an LLM model using OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI()

Create a chain using the prompt and LLM

In [ ]:
# RunnablePassthrough is used take query in Runtime and give to LLM
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# To get final output from the modle
from langchain_core.output_parsers import StrOutputParser

In [ ]:
chain = (
    {"context":retriever ,"question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()    
)

Get a response from the chain via LLM

In [ ]:
chain.invoke("what is Market For Registrant’s Common Equity?")